# Hyperparameter Tuning Tutorial

Machine learning models often have many hyperparameters that need to be tuned to achieve maximal performance (e.g: learning rate, dropout rate, number of layers, layer size, etc) . This motivates the need for hyperparameter tuners that intelligently search the space of hyperparameters for a high performing model. 

To address this, MeTaL supports multiple hyperparameter tuners with an easy to use interface which allows users to streamline the hyperparameter optimization process. This tutorial covers utilizing MeTaL's hyperparameter tuners to tune an EndModel for maximal performance. Currently, two hyperparameter algorithms are supported:

- <b>Random Search</b>
- <b>Hyperband</b>

The tutorial is broken down into the following sections 

1. <b>Set up the Problem and Load the Data</b>
2. <b>Define the Search Space</b>
3. <b>Run Random Search</b>
4. <b>Run Hyperband Search</b>
5. <b>Compare Random Search against Hyperband Search</b>

Let's begin!

## Setup
Before beginning, we first need to make sure that the metal/ directory is on our Python path. If the following cell runs without an error, you're all set. If not, make sure that you've installed snorkel-metal with pip or conda (or that you've added the repo to your path if you're running from source; for example, running source add_to_path.sh from the repository root).

In [176]:
import matplotlib
%load_ext autoreload
%autoreload 2
%matplotlib inline
import metal

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set up the Problem and Load the Data

First let's set up our problem and load our data. For the purposes of this tutorial (and to keep the search process short) we will use the small model introduced in the basic tutorial. 

In [177]:
# Load basic tutorial data
from metal.utils import split_data
import pickle

with open("data/basics_tutorial.pkl", 'rb') as f:
    X, Y, L, D = pickle.load(f)
    
Xs, Ys, Ls, Ds = split_data(X, Y, L, D, splits=[0.8, 0.1, 0.1], stratify_by=Y, seed=123)

Let's define and train our label model like in the basic tutorial.

In [178]:
# Train a the label model
from metal.label_model import LabelModel
label_model = LabelModel(k=2, seed=123)

label_model.train_model(Ls[0], Y_dev=Ys[1], n_epochs=1000, print_every=250, lr=0.01, l2=1e-1)
score = label_model.score((Ls[1], Ys[1]))
scores = label_model.score((Ls[1], Ys[1]), metric=['precision', 'recall', 'f1'])

from metal.label_model.baselines import MajorityLabelVoter

mv = MajorityLabelVoter(seed=123)
scores = mv.score((Ls[1], Ys[1]), metric=['accuracy', 'precision', 'recall', 'f1'])
Y_train_ps = label_model.predict_proba(Ls[0])

Computing O...
Estimating \mu...
[E:0]	Train Loss: 6.036
[E:250]	Train Loss: 0.029
[E:500]	Train Loss: 0.029
[E:750]	Train Loss: 0.029
[E:999]	Train Loss: 0.029
Finished Training
Accuracy: 0.879
        y=1    y=2   
 l=1    178    53    
 l=2    68     701   
Precision: 0.771
Recall: 0.724
F1: 0.746
        y=1    y=2   
 l=1    178    53    
 l=2    68     701   
Accuracy: 0.836
Precision: 0.623
Recall: 0.841
F1: 0.716
        y=1    y=2   
 l=1    207    125   
 l=2    39     629   


Now let's define our EndModel and verify that it successfully runs and achieves a decent score. 

In [179]:
# Train an end model
from metal.end_model import EndModel

end_model_basic = EndModel([1000,2], 
                           train_config={
                               "n_epochs": 5,
                               "l2" : .1,
                               "validation_metric":'f1',
                               "data_loader_config" : {
                                   "batch_size" : 256
                               }
                           }, seed=123)

end_model_basic.train_model((Xs[0], Y_train_ps), dev_data=(Xs[1], Ys[1]))



Network architecture:
Sequential(
  (0): IdentityModule()
  (1): Linear(in_features=1000, out_features=2, bias=True)
)



Saving model at iteration 0 with best score 0.953
[E:0]	Train Loss: 0.491	Dev f1: 0.953


[E:1]	Train Loss: 0.454	Dev f1: 0.908


[E:2]	Train Loss: 0.450	Dev f1: 0.883


[E:3]	Train Loss: 0.449	Dev f1: 0.923


[E:4]	Train Loss: 0.449	Dev f1: 0.879
Restoring best model from iteration 0 with score 0.953
Finished Training
F1: 0.953
        y=1    y=2   
 l=1    225     1    
 l=2    21     753   


Great. Notice that our F1 is around .95. In the sections below we will try to optimize the hyperparameters of this EndModel to achieve an even higher score!

## Define the Search Space

Before starting the hyperparameter tuning process, we need to specify the space of the hyperparameters we're searching. 

For the purposes of this tutorial we search over the following hyperparameters:
- <b>n_epochs</b>: Integer representing the number of epochs to train
- <b>batchnorm</b>: Boolean representing whether to use batch-normalization
- <b>lr</b>: Float representing the learning rate for optimization
- <b>layer_out_dims</b>: The architecture of our neural network

In [180]:
search_space = {
    'seed' : [123],
    'n_epochs': [1],
    'batchnorm' : [True, False],
    'dropout': [0, .1, .2, .3, .4, .5],
    'lr': {'range': [1e-5, 1], 'scale': 'log'},
    'layer_out_dims' : [[1000,10,2], [1000, 100, 2]],
    'print_every': 5,
    'data_loader_config': [{"batch_size": 256, "num_workers": 1}],
}

Here's a breakdown of what each line in the configuration means:

- `'n_epochs': [1, 5, 10],`: This specifies that the hyperparameter tuner may train the model for either 1, 5 or 10 epochs
- `'batchnorm' : [True, False],`: This specifies that a model instantiated by the tuner may have batchnorm as either True or False
- `dropout': [0, .1, .2, .3, .4, .5]`: Like the above, this specifies that the dropout parameter of an instantiated model may be one of 0, .1, .2, .3, .4, or .5
- `'lr': {'range': [1e-5, 1], 'scale': 'log'}`: This specifies that the learning rate of the training of a model may range from 1e-5 to 1, and that the tuner samples the learning rate on a log scale
- `'layer_out_dims' : [[1000,10,2], [1000, 100, 2]]`: This specifies that upon instantiation of the model, the structure of the fully connected network can either be [1000, 10, 2] or [1000, 100, 2]; in the latter case, this means the network takes a 1000 dimensional input, has a hidden layer with 100 features and an output layer with 2 classes
- `'print_every': 5`: This specifies that the model should print status updates every 5 iterations of training.
- `'data_loader_config': [{"batch_size": 256, "num_workers": 1}],`: This specifies to use a batch of 256 for optimization

Now that our search space is defined, let's start optimizing hyperparameters!

## Run Random Search

While simple, random search has proven to be a powerful and efficient algorithm for tuning hyperparameters (see http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf for why). Let's use the RandomSearch tuner to find a good set of hyperparameters for our EndModel. Note that although we only do hyperparameter optimization for the EndModel, the tuners may also be used to do hyperparameter optimization for LabelModels.

To start, let's import the RandomSearchTuner and instantiate our RandomSearchTuner to optimize an EndModel model.

In [181]:
from metal.tuners.random_tuner import RandomSearchTuner
rs_tuner = RandomSearchTuner(EndModel, seed=123, validation_metric="f1")

Next let's define our training and validation datasets.

In [182]:
train_args = [(Xs[0], Y_train_ps)]
X_dev, Y_dev = Xs[1], Ys[1]

And just like that we're prepped to launch our random search! Performing the search is just as easy and requires just a single call to the `search` function.

Most of the arguments to the `search` function below are self explanatory, but there are a couple of key arguments to watch out for:
- `max_search` : This specifies the number of configurations to search over. As it is set to 20 below, this means we search over 20 random models and return the best one
- `verbose`: This specifies whether the tuner should be verbose or not and can be used to turn on/off the its logging feature

In [183]:
best_rs_model = rs_tuner.search(search_space, (X_dev, Y_dev), train_args=train_args, max_search=20, verbose=False)
rs_tuner.best_score

1.0

Awesome, our best random search model achieves an F1 of ~.98 which outperforms the model we had previously (F1 ~ .95). Can we do even better than random search by either attaining the same accuracy faster or achieving a higher score? The following section walks through using the <b>Hyperband</b> tuner, which recent research has shown to be more efficient than random search.

## Run Hyperband Search

While random search performs surprisingly well, we can be more efficient if we adaptively allocate more compute resources for configurations that perform well than to those that don't. For example if a configuration seems to yield a really poor model after the first epoch of training, it's unlikely it'll perform well even after more training, so we can early-terminate the training of this configuration to save compute. This is the core idea behind the <b>Hyperband</b> algorithm which recent research has shown to outperform various algorithms including random search. (See https://arxiv.org/abs/1603.06560 if interested!)

Running Hyperband is just as easy as running random search. Let's import the HyperbandTuner and instantiate it. 

Note that there is one extra argument to initialize the HyperbandTuner:
- `hyperband_epochs_budget`: This specifies the number of total epochs of training the tuner can perform in its search for a performant model. This is used to create the Hyperband search schedule.

In [184]:
from metal.tuners.hyperband_tuner import HyperbandTuner
hb_tuner = HyperbandTuner(EndModel, hyperband_epochs_budget=200, seed=123, validation_metric="f1")

|           Hyperband Schedule          |
Table consists of tuples of (num configs, num_resources_per_config)which specify how many configs to run andfor how many epochs. 
Each bracket starts with a list of random configurations which is successively halved according the schedule.
See the Hyperband paper (https://arxiv.org/pdf/1603.06560.pdf) for more details.
-----------------------------------------
Bracket 0: (9, 2) (3, 8) (1, 26)
Bracket 1: (3, 8) (1, 26)
Bracket 2: (3, 26)
-----------------------------------------


We can launch the Hyperband search process using the same `search` call. Note that since the Hyperband schedule already limits the amount of compute we do, we don't have to set the `max_search` argument.

In [185]:
best_hb_model = hb_tuner.search(search_space, (X_dev, Y_dev), train_args=train_args, verbose=False)
hb_tuner.best_score

TypeError: config_generator() missing 1 required positional argument: 'rng'

Awesome, we achieved an F1 ~.985, which beat our initial score of F1 ~.95 and essentially matches the score achieved by random search (F1 ~.98). The next section will compare the performances of random search and Hyperband using the logged data.

## Compare Random Search against Hyperband Search

During `search`, MeTaL hyperparameter tuners track useful statistics. Currently these include
* Time elapsed
* Achieved score
* Configuration for score

We will analyze these statistics to compare random search against Hyperband. 

First, let's extract the statistics captured by our tuners.

In [ ]:
hb_stats = hb_tuner.run_stats
rs_stats = rs_tuner.run_stats

Let's take a look at a few of the collected datapoints

In [ ]:
hb_stats[:3]

The `run_stats` property is a list of datapoints, where each datapoint (captured by a dictionary) specifies the current elapsed timestamp, the score achieved for that timestamp and the hyperparameter configuration for the score.

Let's extract the best score achieved per timestamp.

In [ ]:
def gather_best_scores_from_stats(stats):    
    res = []
    best_score = float("-inf")
    for stat in stats:
        best_score = max(best_score, stat["score"])
        time_elapsed = stat["time_elapsed"]
        res.append({"best_score" : best_score, "time_elapsed" : time_elapsed})
    return res

hb_best_score_stats = gather_best_scores_from_stats(hb_stats)
rs_best_score_stats = gather_best_scores_from_stats(rs_stats)

Now that we have our score data, let's compare the performances of random search and hyperband by plotting the best scores they achieve across time!

In [ ]:
import matplotlib.pyplot as plt

def plot_score_vs_time(rs_data, hb_data, logscale=False):
    plt.cla()
    
    if logscale:
        plt.xscale("log")
    
    xs_hyperband = [x["time_elapsed"] for x in hb_data]
    ys_hyperband = [x["best_score"] for x in hb_data]
    xs_rs = [x["time_elapsed"] for x in rs_data]
    ys_rs = [x["best_score"] for x in rs_data]
    
    plt.plot(xs_hyperband, ys_hyperband, label="Hyperband")
    plt.plot(xs_rs, ys_rs, label="RandomSearch")
    
    plt.xlabel("Time (s)")
    plt.ylabel("F1 Score Achieved")
        
    plt.legend(loc="best")
    
plot_score_vs_time(rs_best_score_stats, hb_best_score_stats)

Interesting! Our plot shows that Hyperband finds a good solution faster than random search. 

From the plot above, we see that Hyperband finds its best solution at around time 10s, whereas it takes random search around 50-75s to achieve a comparable score. 

And that wraps up the hyperparameter tuning tutorial! We hope that MeTaL's hyperparameter tuners are useful for your applications and make hyperparameter tuning enjoyable. Happy tuning!